# Logistic Regression

## Plan

1. From linear combinations to predicting the probability.
2. How to find coefficients $\theta$? Maximum likelihood.
3. Gradient descent implementation
4. Regularization

### Linear combinations >> probability

The idea is similar to Linear Regression - use $X*\theta$ for prediction. But now we say that we want to predict the probability - the value $P(y | X) \in [0 ... 1]$

$$
P(y = 1|x;\theta) = h_\theta(x) \\
P(y = 0|x;\theta) = 1 - h_\theta(x)
$$

We need some $g(\theta^T * X)$
$$
h_\theta(X) = g(\theta^T * X) = g(z), \text{ where } z = \theta^T * X \in [-\infty ... +\infty]\\
$$
In other words, we need:
$$
\theta^T * X \in [-\infty ... +\infty] =>> P(y | X) \in [0 ... 1]
$$

- transform $\theta^T * X$ into $[0 ... 1]$
- or find some $f(P)$ that returns $[-\infty ... +\infty]$ and derive $P(y | X)$ from it.

### Odds and log(odds)

$$
P(y = 1|x;\theta) = h_\theta(x) \in [0 ... 1]\\
Odds(y = 1) = \frac {P(y = 1)} {P(y = 0)} = \frac {P(y = 1)} {1 - P(y = 1)}  \in [0 ... +\infty] \\
\log{Odds(y = 1)} = \log{\frac {P(y = 1)} {1 - P(y = 1)}} \in [-\infty ... +\infty]
$$
We found a function $f(P)$ that returns values $[-\infty ... +\infty]$. So, now we can make aт assumption that
$$
\theta^T * X = \log{\frac {P(y = 1)} {1 - P(y = 1)}} \\
$$
Then:
$$
P(y = 1 | X; \theta) = h_\theta(X) = \frac {1} {1 + e^{-(\theta^T * X)}}
$$

Now the question is - how to find the best $\theta$

### Likelihood



$$
\text{For } y = 1: p(y|x,\theta) = h_\theta(x) \\
\text{For } y = 0: p(y|x,\theta) = 1 - h_\theta(x) \\
$$
Same thing with one formula:
$$
p(y|x,\theta) = (h_\theta(x))^y (1 - h_\theta(x))^{1 - y} \\ 
p(y|x,\theta) = (\frac {1} {1 + e^{-(\theta^T * X)}})^y (1 - \frac {1} {1 + e^{-(\theta^T * X)}})^{1 - y}
$$

We need the most likely parameters theta. The LIKELIHOOD of parametrs $\theta$ is basically the same as the probability of y but with $\theta$ as a variable. Probability of all y values is the product of all probabilities (joint probability).

$$
L(\theta) = p(\vec{y}|X;\theta) \\
= \prod_{i=1}^{n} p(y^{(i)}|x^{(i)};\theta) \\
= \prod_{i=1}^{n} (h_\theta(x^{(i)}))^{y^{(i)}} (1 - h_\theta(x^{(i)}))^{1 - y^{(i)}} \\
= \prod_{i=1}^{n} (\frac {1} {1 + e^{-(\theta * x^{(i)})}})^y (1 - \frac {1} {1 + e^{-(\theta * x^{(i)})}})^{1 - y}
$$

Logistic function is monotonic, which allows us to maximize the logarythm of this product. We replace Linelihood with LOG of Likelihood. Then PRODUCT will transform to SUMM.

$$
l(\theta) = \log L(\theta) \\
= \sum_{i=1}^n y^{(i)} \log h(x^{(i)}) + (1 - y^{(i)})log(1 - h(x^{(i)}) \\
= \sum_{i=1}^n y^{(i)} \log \frac {1} {1 + e^{-(\theta * x^{(i)})}} + (1 - y^{(i)})log(1 - \frac {1} {1 + e^{-(\theta * x^{(i)})}}) \\
$$

But now the value depends on the dataset size - sum from 1 to n. So, for convenience we need to scale the whole equation:

$$
l(\theta) = \frac{1}{n} * \sum_{i=1}^n y^{(i)} \log \frac {1} {1 + e^{-(\theta * x^{(i)})}} + (1 - y^{(i)})log(1 - \frac {1} {1 + e^{-(\theta * x^{(i)})}}) \\
$$

And this thing we want to maximize

### Derivative of Likelihood function

In advance we can say:

$$
g(z) = \frac{1}{1 + e^{-z}} \\
$$
Derivative of a fraction of two functions
$$
\frac{d}{dz} g(z) = \frac{d}{dz} \frac{1}{f(z)} = \\
= \frac{0 * (1 + e^{-z}) - 1 * \frac{d}{dz} (1 + e^{-z}) )}{(1 + e^{-z})^2} = \\
= \frac{-(-e^{-z})}{(1 + e^{-z})^2} = \frac{e^{-z} + 1 - 1}{(1 + e^{-z})^2} = \\
= (1 - \frac{1}{1 + e^{-z}}) * \frac{1}{1 + e^{-z}} = \\
= g(z) * (1 - g(z)) \\
$$
Coming back to $h_{\theta}(x)$ we need to add $\frac{dz}{dx} = \frac{d\theta^T*X}{dx} = X$ to the end
$$
\text{1. } \frac {\partial{}} {\partial{\theta}} h_\theta(X) = h_\theta(X) * (1 - h_\theta(X)) * X = \frac {1} {1 + e^{-(\theta^T * X)}} * (1 - \frac {1} {1 + e^{-(\theta^T * X)}}) * X \\
\text{2. } \frac {\partial{}} {\partial{\theta}} \log{h_\theta(X)} = \frac{1}{h_\theta(X)} * h_\theta(X) * (1 - h_\theta(X) * X) = (1 - h_\theta(X)) * X \\
\text{3. } \frac {\partial{}} {\partial{\theta}} \log{\left(1 - h_\theta(X) \right)} = \frac{1}{1 - h_\theta(X)} * \left[ 0 - h_\theta(X) * (1 - h_\theta(X)) * X \right]  =  \\
= \frac{1}{1 - h_\theta(X)} * 0 - \frac{1}{1 - h_\theta(X)} * h_\theta(X) * (1 - h_\theta(X)) * X = \\
= - h_\theta(X) * X\\
$$

Finally:

$$
\frac {\partial{}} {\partial{\theta}} l(\theta) = \frac{1}{n} * \sum_{i = 1}^{n} X * \left( y^{(i)} - h_\theta(X) \right) = \frac{1}{n} * \sum_{i = 1}^{n} X * \left( y^{(i)} - \frac {1} {1 + e^{-(\theta * x^{(i)})}} \right)
$$

Gradient solution:

$$
\theta := \theta + \alpha * \frac {\partial{}} {\partial{\theta}} l(\theta) \\
$$

### Regularization

Works the same as for Linear Regression:

$$
\text{old l}(\theta) = \frac{1}{n} * \sum_{i=1}^n y^{(i)} \log \frac {1} {1 + e^{-(\theta * x^{(i)})}} + (1 - y^{(i)})log(1 - \frac {1} {1 + e^{-(\theta * x^{(i)})}}) \\
\text{new l}(\theta) = \text{old l}(\theta) + \lambda * ||\theta||_k^k
$$

For k=2:
$$
\text{new l}(\theta) = \text{old l}(\theta) + \lambda * ||\theta||_2^2 \\
\frac {\partial{}} {\partial{\theta}} \text{new l}(\theta) = \frac {\partial{}} {\partial{\theta}} \text{old l}(\theta) + 2 * \lambda * \theta = \\
= \frac{1}{n} * \sum_{i = 1}^{n} X * \left( y^{(i)} - \frac {1} {1 + e^{-(\theta * x^{(i)})}} \right) + 2 * \lambda * \theta
$$

$$
\theta := \theta + \alpha * \frac {\partial{}} {\partial{\theta}} \text{new l}(\theta) = \\
\theta := \theta + \alpha * \left( \frac{1}{n} * \sum_{i = 1}^{n} X * \left( y^{(i)} - \frac {1} {1 + e^{-(\theta * x^{(i)})}} \right) + 2 * \lambda * \theta \right)
$$

## Interpretation

In [8]:
import numpy as np
from numpy import linalg as la
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 50)

RANDOM_STATE = 42

I'll be using Spaceship Titanic dataset from [Kaggle competition](https://www.kaggle.com/competitions/spaceship-titanic)

Data description:
* PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
* HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
* CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
* Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
* Destination - The planet the passenger will be debarking to.
* Age - The age of the passenger.
* VIP - Whether the passenger has paid for special VIP service during the voyage.
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
* Name - The first and last names of the passenger.
* Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

In [10]:
data = pd.read_csv('datasets/spaceship_titanic/train.csv')

In [11]:
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [12]:
X = data[['Age', 'FoodCourt', 'ShoppingMall', 'VRDeck', 'Spa']]
X = X.dropna()
y = data['Transported']
y = y[X.index].astype('int').values

# Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)
ones = np.ones(len(X))[:, None]
X = np.concatenate([ones, X], axis=1)

print(X[:5])
print(y[:5])

[[ 1.          0.69947548 -0.28543451 -0.28469585 -0.26876547 -0.27612744]
 [ 1.         -0.33398833 -0.27998983 -0.24397213 -0.22947464  0.20090496]
 [ 1.          2.00852964  1.87791977 -0.28469585 -0.22500978  5.55861321]
 [ 1.          0.28608996  0.49073538  0.31964411 -0.09642163  2.6164789 ]
 [ 1.         -0.88516903 -0.24308698 -0.0387246  -0.26697952  0.21480755]]
[0 1 0 0 1]


In [13]:
import numpy as np

class LogRegression:
    def __init__(self, alpha=0.01, n_iterations=1000):
        self.alpha = alpha
        self.n_iterations = n_iterations
        return
    
    def hypothesis(self, theta, X):
        return 1 / (1 + np.exp(-theta @ X.T))
    
    def fit(self, X, y):
        self.theta = np.zeros(X.shape[1])
        hypothesis = self.hypothesis(self.theta, X)
        errors = y - hypothesis
        grad = X.T @ errors
        self.theta += self.alpha * grad
        
        for _ in range(self.n_iterations):
            hypothesis = self.hypothesis(self.theta, X)
            errors = y - hypothesis
            grad = X.T @ errors / len(X)
            self.theta += self.alpha * grad
        return self.theta
    
    def predict(self, X):
        return self.hypothesis(self.theta, X)
    
    
model = LogRegression(alpha=0.01, n_iterations=1000)
thetas = model.fit(X, y)
predictions_proba = model.predict(X)
predictions = np.where(predictions_proba > 0.5, 1, 0)

print(f'Thetas: {thetas}')
print(f'Accuracy: {balanced_accuracy_score(y, predictions)}')

Thetas: [-1.5329799  -1.27043347  2.51789466  0.32548171 -7.48842278 -8.14083801]
Accuracy: 0.6844583036745429


With regularization:

In [15]:
import numpy as np

class LogRegressionL2:
    def __init__(self, alpha=0.01, n_iterations=1000, c=0):
        self.alpha = alpha
        self.n_iterations = n_iterations
        self.lmbd = c
        return
    
    def hypothesis(self, theta, X):
        return 1 / (1 + np.exp(-theta @ X.T))
    
    def fit(self, X, y):
        self.theta = np.zeros(X.shape[1])
        hypothesis = self.hypothesis(self.theta, X)
        errors = y - hypothesis
        grad = X.T @ errors
        self.theta += self.alpha * grad
        
        for _ in range(self.n_iterations):
            dummy_theta = self.theta.copy()
            dummy_theta[0] = 0
            hypothesis = self.hypothesis(self.theta, X)
            errors = y - hypothesis
            grad = X.T @ errors / len(X)
            self.theta += self.alpha * (grad + self.lmbd * dummy_theta)
        return self.theta
    
    def predict(self, X):
        return self.hypothesis(self.theta, X)
    
    
model = LogRegressionL2(alpha=0.01, n_iterations=1000, c=0.01)
thetas = model.fit(X, y)
predictions_proba = model.predict(X)
predictions = np.where(predictions_proba > 0.5, 1, 0)

print(f'Thetas: {thetas}')
print(f'Accuracy: {balanced_accuracy_score(y, predictions)}')

Thetas: [-1.58048501 -1.47775439  2.76464137  0.36479095 -8.29152851 -9.01096115]
Accuracy: 0.6823934189000977
